In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [2]:
# Function to load images from directory
def load_images_from_folder(folder, is_gray = True):
    images = []
    for filename in os.listdir(folder):
        img = io.imread(os.path.join(folder,filename)).astype(np.uint8)
        if img is not None:
            img = img/255
            if is_gray:
                images.append(color.rgb2gray(img))
            else:    
                images.append(img)
    return images


In [3]:
# Load images from folders A and B
folder_A = 'testval/A'
folder_B = 'testval/B'
folder_label = 'testval/label'

In [4]:
images_A = load_images_from_folder(folder_A)
images_B = load_images_from_folder(folder_B)
labels = load_images_from_folder(folder_label, is_gray = False)

In [5]:
images_A = np.array(images_A)
images_B = np.array(images_B)
labels = np.array(labels)

In [6]:

def calculate_jaccard_index(pred_mask, true_mask):
    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()
    jaccard_index = intersection / union
    return jaccard_index

def calculate_jaccard_index_avg(pred_mask, true_mask):
    arr = []
    for i in range(len(pred_mask)):
        intersection = np.logical_and(pred_mask[i], true_mask[i]).sum()
        union = np.logical_or(pred_mask[i], true_mask[i]).sum()
        if union != 0:
            arr.append(intersection / union)
        else:
            arr.append(1)
    
    jaccard_index = np.array(arr).mean()        
    return jaccard_index

In [7]:
def change_detection_cva(images_time1, images_time2):
    if images_time1.shape != images_time2.shape:
        raise ValueError("Input arrays must have the same shape")

    change_images = np.zeros_like(images_time1, dtype=np.uint8)

    change_vectors = np.abs(images_time2 - images_time1)

    for i in range(images_time1.shape[0]):
        for j in range(images_time1.shape[1]):
            magnitude = np.linalg.norm(change_vectors[i, j])
            change_images[i, j] = magnitude
            
    return change_images

In [8]:
def image_difference(image_A, image_B):
    diff = np.abs(image_A - image_B)
    return diff

In [9]:
cva_mask = change_detection_cva(images_A, images_B)

In [18]:
cva_mask_thresholded = cva_mask > 2

In [19]:
jaccard_index = calculate_jaccard_index(cva_mask_thresholded, labels)
jaccard_index_avg = calculate_jaccard_index_avg(cva_mask_thresholded, labels)
print("For CVA Jaccard Index:", jaccard_index)
print("For CVA Jaccard Index Avg:", jaccard_index_avg)

For CVA Jaccard Index: 0.020393760595666836
For CVA Jaccard Index Avg: 0.641236050600443


In [20]:
change_mask = image_difference(images_A, images_B)

In [21]:
thresholds = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]

for th in thresholds:
    change_mask_thresholded = change_mask > th
    jaccard_index = calculate_jaccard_index(change_mask_thresholded, labels)
    jaccard_index_avg = calculate_jaccard_index_avg(change_mask_thresholded, labels)
    print(f"Threshold:{th} | Our jaccard:{jaccard_index} | Avg jaccard:{jaccard_index_avg}")
    # print(confusion_matrix(labels, change_mask_thresholded))
    print("===========================================================================================================")

Threshold:0.05 | Our jaccard:0.009665183362494709 | Avg jaccard:0.007285447537951842
Threshold:0.1 | Our jaccard:0.013243448669261908 | Avg jaccard:0.008193102431354878
Threshold:0.15 | Our jaccard:0.016578066580712224 | Avg jaccard:0.00845200594198163
Threshold:0.2 | Our jaccard:0.019855131856503342 | Avg jaccard:0.012745709804122011
Threshold:0.25 | Our jaccard:0.02527881358241676 | Avg jaccard:0.04400281306142276
Threshold:0.3 | Our jaccard:0.03143803636236002 | Avg jaccard:0.10286201736443286
Threshold:0.35 | Our jaccard:0.030560845883413623 | Avg jaccard:0.17948671869021013
Threshold:0.4 | Our jaccard:0.01971641641914416 | Avg jaccard:0.30463669061537996
Threshold:0.45 | Our jaccard:0.009962868911351582 | Avg jaccard:0.4948490940588774
Threshold:0.5 | Our jaccard:0.004774716353064877 | Avg jaccard:0.6505135688632608
